In [81]:
from engine_manager import EngineManager
from features_models import FeaturesCollapsedSctn, FeaturesCollapsedSttc
from features_queries import UpsertQuerySctn, UpsertQuerySttc
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects.postgresql import insert
import tomllib

import os

import pandas as pd

In [110]:
engine_manager = None

__loc = os.getcwd()
with open(os.path.join(__loc, 'db_cfg.toml'), 'rb') as cfg:
    feats_db_cfg = tomllib.load(cfg)['DATABASE-local']
    
    username = feats_db_cfg['USERNAME']
    password = feats_db_cfg['PASSWORD']
    host = feats_db_cfg['HOST']
    database = feats_db_cfg['DATABASE']
    port = feats_db_cfg['PORT']
    
    engine_manager = EngineManager(
        username=username,
        password=password,
        host=host,
        database=database,
        port=port
    )

engine = engine_manager.engine()

In [8]:
feats = ('rolloff', 'centroid', 'bandwidth',
         'contrast', 'rms', 'flux',
         'dynamic_range', 'instrumentalness')
n_sections = 8

dtypes = {f'collapsed_{feat}_{section}': float for feat in feats for section in range(1, n_sections + 1)}

dtypes = {**dtypes, 'track_name': str, 'major': str, 'minor': str, 'bpm': int}

large_df = pd.read_csv(
    filepath_or_buffer='test_large.csv',
    dtype=dtypes
).drop_duplicates(subset=['track_name'], keep='first')

large_df.loc[:, 'major'] = large_df.loc[:, 'major'].str[:-6]
large_df.loc[:, 'minor'] = large_df.loc[:, 'minor'].str[:-6]

In [112]:
bigboy = large_df.filter(regex=rf'(.*{feats[0]}.*)|(track_name)', axis=1).melt(id_vars=['track_name'], var_name='section', value_name=feats[0])
bigboy.loc[:, 'section'] = bigboy.loc[:, 'section'].str[-1]

for feat in feats[1:]:
    temp = large_df.filter(regex=rf'(.*{feat}.*)|(track_name)', axis=1).melt(id_vars=['track_name'], var_name='section', value_name=feat)
    temp.loc[:, 'section'] = temp.loc[:, 'section'].str[-1]
    bigboy = bigboy.merge(temp, on=['track_name', 'section'])

bigboy.sort_values(by=['track_name', 'section'])

bigboy = bigboy.rename(columns={
    'track_name': 'track_id',
    'rolloff': 'spctrl_rlf',
    'centroid': 'spctrl_cntrd',
    'bandwidth': 'spctrl_bw',
    'contrast': 'spctrl_cntrst',
    'flux': 'spctrl_flux',
    'dynamic_range': 'dnmc_rng',
    'instrumentalness': 'instrmntlns'
})

In [ ]:
bigboy[bigboy['track_id'] == '2841884242']

,track_id,section,spctrl_rlf,spctrl_cntrd,spctrl_bw,spctrl_cntrst,rms,spctrl_flux,dnmc_rng,instrmntlns
0,2841884242,1,10060.565832,3565.457222,1818.559611,41.240259,-17.151496,292.488304,8.042248,0.574844
11904,2841884242,2,11052.613166,3532.547071,1985.987341,42.122718,-16.320020,279.914631,7.747415,0.709188
23808,2841884242,3,10982.566923,3413.976909,1954.192328,41.557201,-15.448866,282.710827,8.023149,0.767459
35712,2841884242,4,11068.573070,3430.423335,2043.204034,40.740297,-14.577069,288.297029,8.049490,0.681098
47616,2841884242,5,12313.762207,4085.299547,1960.247685,40.052136,-16.172295,280.301937,7.772585,0.721726
59520,2841884242,6,10748.045008,3566.904236,2085.506213,40.566279,-14.753502,274.760741,7.810733,0.661584
71424,2841884242,7,10655.768899,3157.456154,2252.558250,40.644848,-14.274145,272.812476,7.662453,0.819256
83328,2841884242,8,10872.880285,3619.734180,2129.764239,40.268639,-15.320971,280.682818,7.830424,0.583565


In [116]:
session_maker = sessionmaker(engine)

with session_maker() as session:
    # upsert_query_sttc = UpsertQuerySttc(
    #     table=FeaturesCollapsedSttc,
    #     track_id='1234567890',
    #     bpm=12.3,
    #     keymjr='C',
    #     keymnr='a')

    session.execute(insert(FeaturesCollapsedSctn), bigboy.to_dict(orient='records'))

    # session.execute(upsert_query_sttc.upsert_id_feats)

    session.commit()

Imports

In [1]:
from engine_manager import EngineManager
from features_models import FeaturesCollapsedSctn, FeaturesCollapsedSttc
from features_queries import UpsertQuerySctn, UpsertQuerySttc
from sqlalchemy.orm import sessionmaker
import tomllib
import os
import pandas as pd
from features_queries import SelectQuerySctn, SelectQuerySttc

Establishing a connection (use DATABASE-local if running locally on server, otherwise DATABASE-remote) -> Remember to change `db_cfg.toml`

In [2]:
engine_manager = None

__loc = os.getcwd()
with open(os.path.join(__loc, 'db_cfg.toml'), 'rb') as cfg:
    feats_db_cfg = tomllib.load(cfg)['DATABASE-local']
    
    username = feats_db_cfg['USERNAME']
    password = feats_db_cfg['PASSWORD']
    host = feats_db_cfg['HOST']
    database = feats_db_cfg['DATABASE']
    port = feats_db_cfg['PORT']
    
    engine_manager = EngineManager(
        username=username,
        password=password,
        host=host,
        database=database,
        port=port
    )

engine = engine_manager.engine()

Shared utilities :)

In [3]:
select_query_sctn_all = SelectQuerySctn(FeaturesCollapsedSctn)
select_query_sttc_all = SelectQuerySttc(FeaturesCollapsedSttc)

session_maker = sessionmaker(engine)

sctn_feats = None
sttc_feats = None

TypeError: SelectQuerySctn.__init__() missing 1 required positional argument: 'where_col_equal'

Check if the record exists

In [9]:
select_query_sttc_one = SelectQuerySttc(table=FeaturesCollapsedSttc, where_col_equal=(FeaturesCollapsedSttc.track_id, '2841884242'))
select_query_sctn_one = SelectQuerySctn(table=FeaturesCollapsedSctn, where_col_equal=(FeaturesCollapsedSctn.track_id, '2841884242'))

with engine.connect() as conn:
    sctn_feats = pd.read_sql(select_query_sctn_one.select_id_feats, conn)
    sttc_feats = pd.read_sql(select_query_sttc_one.select_id_feats, conn)

if sttc_feats.shape[0] == 0:
    # Record doesn't already exist, need to featurize
    pass
else:
    # Record exists, can use it...
    sctn_feats = sctn_feats.rename(columns={'track_id': 'track_name'})
    sttc_feats = sttc_feats.rename(columns={'track_id': 'track_name'})

    feats = ('spctrl_rlf', 'spctrl_cntrd', 'spctrl_bw',
            'spctrl_cntrst', 'rms', 'spctrl_flux',
            'dnmc_rng', 'instrmntlns')

    id_cols = ('track_name', 'section')

    bigguy = sctn_feats.loc[:, id_cols + (feats[0],)].pivot(index='track_name',
                                                        columns='section',
                                                        values=feats[0]).rename(
                                                            columns={i: f'{feats[0]}_{i}' for i in range(1, n_sections+1)}
                                                            ).reset_index()

    for feat in feats[1:]:
        temp = sctn_feats.loc[:, id_cols + (feat,)].pivot(index='track_name',
                                                        columns='section',
                                                        values=feat).rename(
                                                            columns={i: f'{feat}_{i}' for i in range(1, n_sections+1)}
                                                            ).reset_index()
        bigguy = bigguy.merge(temp, on=['track_name'])

    bigguy = bigguy.merge(sttc_feats, on=['track_name'])

In [10]:
bigguy

,track_name,spctrl_rlf_1,spctrl_rlf_2,spctrl_rlf_3,spctrl_rlf_4,spctrl_rlf_5,spctrl_rlf_6,spctrl_rlf_7,spctrl_rlf_8,spctrl_cntrd_1,...,instrmntlns_2,instrmntlns_3,instrmntlns_4,instrmntlns_5,instrmntlns_6,instrmntlns_7,instrmntlns_8,bpm,keymjr,keymnr
0,2841884242,10060.565832,11052.613166,10982.566923,11068.57307,12313.762207,10748.045008,10655.768899,10872.880285,3565.457222,...,0.709188,0.767459,0.681098,0.721726,0.661584,0.819256,0.583565,80,C,F


Obtaining all records from both tables and constructing wide table (68 columns with all rows)

In [ ]:
with engine.connect() as conn:

    sctn_feats = pd.read_sql(select_query_sctn_all.select_id_feats, conn)
    sttc_feats = pd.read_sql(select_query_sttc_all.select_id_feats, conn)

sctn_feats = sctn_feats.rename(columns={'track_id': 'track_name'})
sttc_feats = sttc_feats.rename(columns={'track_id': 'track_name'})

feats = ('spctrl_rlf', 'spctrl_cntrd', 'spctrl_bw',
         'spctrl_cntrst', 'rms', 'spctrl_flux',
         'dnmc_rng', 'instrmntlns')

id_cols = ('track_name', 'section')

bigguy = sctn_feats.loc[:, id_cols + (feats[0],)].pivot(index='track_name',
                                                       columns='section',
                                                       values=feats[0]).rename(
                                                           columns={i: f'{feats[0]}_{i}' for i in range(1, n_sections+1)}
                                                        ).reset_index()

for feat in feats[1:]:
    temp = sctn_feats.loc[:, id_cols + (feat,)].pivot(index='track_name',
                                                     columns='section',
                                                     values=feat).rename(
                                                         columns={i: f'{feat}_{i}' for i in range(1, n_sections+1)}
                                                        ).reset_index()
    bigguy = bigguy.merge(temp, on=['track_name'])

bigguy = bigguy.merge(sttc_feats, on=['track_name'])

In [19]:
import socketserver
import soundfile as sf
import pickle
import numpy as np

In [ ]:
sctn_feats = None

class MyTCPHandler(socketserver.StreamRequestHandler):
    """
    The request handler class for our server.

    It is instantiated once per connection to the server, and must
    override the handle() method to implement communication to the
    client.
    """

    def handle(self):
        self.data = self.rfile.readline(2**12).rstrip()
        
        # np.fromstring()
        global sctn_feats 
        sctn_feats = np.frombuffer(self.data)
        
        # self.wfile.write(self.data.upper())

if __name__ == "__main__":
    HOST, PORT = '192.168.2.200', 9999

    # Create the server, binding to localhost on port 9999
    with socketserver.TCPServer((HOST, PORT), MyTCPHandler) as server:
        # Activate the server; this will keep running until you
        # interrupt the program with Ctrl-C
        server.serve_forever()


KeyboardInterrupt: 